<a href="https://colab.research.google.com/github/PatrickRuan/AI_2025/blob/main/01a_ATM_Advanced_web.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 使用 Ngrok 建立 Web Base 的程式存取架構

我們的這份程式（`01a.ATM_Advanced_web.ipynb`）屬於 **Web base 的版本**，也就是會啟動一個網頁伺服器，讓使用者可以透過網頁與程式互動。

但一般來說，我們電腦上的程式（例如 Flask 啟動的網站）只能在本地的 `localhost` 存取，其他人無法直接連線。

---

### 🔧 解決方法：使用 [ngrok](https://ngrok.com/)

Ngrok 的功能是：
- 幫我們的本地伺服器（例如 `localhost:5000`）建立一個**公開的網址**（free URL）
- 把這個網址對應轉送到我們電腦上指定的 port
- 讓外部電腦可以透過這個網址使用我們的服務

---

### 🔁 架構流程圖解：

如下圖所示（請參考投影片中的圖）：

- **粉紅色區塊（程式 + 本地電腦接口）**：這是我們在電腦上寫的程式，相當於 server，負責主要的處理與計算。
- **NGROK 的 free URL**：相當於一座橋樑，把外部世界與本地的程式接起來。
- **右側的其他電腦**：是從網路上要來存取我們服務的使用者或設備。

簡單來說：  
> **Ngrok 就是把我們電腦裡 localhost:xxxx 這個服務包裝成一個真正可以從外部打開的網址！**

---

✅ 這樣的方式非常適合開發階段、展示階段，或是給別人測試我們寫的 web 程式用。不需要租用雲端主機，也不需要自己設防火牆。



In [ ]:
!pip install pyngrok  # 用 ! 就是用我們的 terminal, cli, cmd

from flask import Flask, request
from pyngrok import ngrok, conf
from getpass import getpass

# 輸入 ngrok authtoken
auth_token = getpass("請輸入你的 ngrok authtoken：")

# 設定 authtoken
conf.get_default().auth_token = auth_token # conf 是 pyngrok 的一個物件

# 開啟 ngrok 隧道
public_url = ngrok.connect(5000)
print(f"🌍 Ngrok 公開網址: {public_url}")

# 建立 Flask app
app = Flask(__name__)

accounts_dict = {
    'user1234': {'password': '5678', 'name': '小明'},
    'user5678': {'password': '1234', 'name': '阿美'}
}
# HTML
login_form = '''
<h2>ATM 登入系統</h2>
<form method="POST">
    帳號：<input type="text" name="username"><br>
    密碼：<input type="password" name="password"><br>
    <input type="submit" value="登入">
</form>
'''

@app.route('/', methods=['GET', 'POST'])
def login():
    if request.method == 'POST':
        username = request.form.get('username')
        password = request.form.get('password')

        if username in accounts_dict and password == accounts_dict[username]['password']:
            return f"<h3>✅ 登入成功！歡迎 {accounts_dict[username]['name']}！</h3>"
        else:
            return "<h3>❌ 帳號或密碼錯誤！</h3>" + login_form
    return login_form

# 啟動 Flask 伺服器
app.run(host='0.0.0.0', port=5000)


請輸入你的 ngrok authtoken：··········
🌍 Ngrok 公開網址: NgrokTunnel: "https://4cfb-34-122-49-210.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [10/Apr/2025 07:45:48] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/Apr/2025 07:45:49] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [10/Apr/2025 07:45:55] "POST / HTTP/1.1" 200 -


## Flask 程式結構介紹：建立一個簡單的登入系統

我們在這裡使用 Python 的 Flask 套件來建立一個 **網頁應用程式**（Web App）。

---

### 🔁 基本流程

1. 使用者打開網址（由 ngrok 提供的公開網址）
2. Flask 伺服器收到請求，根據網址與方法（GET 或 POST）來決定要執行什麼功能
3. 如果是 GET：顯示登入表單  
   如果是 POST：接收帳號密碼並進行比對

---

### ✨ 重點語法說明

#### 🔹 `@app.route('/', methods=['GET', 'POST'])`

- `@app.route(...)` 是一個「裝飾器」，它的意思是：
  
  👉「當有人造訪 `/` 這個網址時，要執行下面定義的函式（`login()`）」

- `methods=['GET', 'POST']` 表示這個網址支援兩種請求方式：
  - **GET**：通常是打開網頁（沒送資料）
  - **POST**：是從表單送出資料（帳號密碼）

---

#### 🔹 `def login():`

這個函式就是處理登入邏輯的地方：
- 如果是 **POST**，表示使用者送出表單，我們就取出帳號密碼進行比對。
- 如果是比對成功，就回傳歡迎訊息。
- 如果失敗，回傳錯誤訊息＋登入表單。
- 如果只是打開網頁（GET），就顯示登入表單。

---

#### 🔹 `app.run(host='0.0.0.0', port=5000)`

- 這一行的意思是「啟動 Flask 伺服器」
- `host='0.0.0.0'`：代表開放給所有 IP 存取（讓 ngrok 可以連進來）
- `port=5000`：是我們開的伺服器 port，跟 ngrok 那行 `ngrok.connect(5000)` 對應

---

### ✅ 小總結

| Flask 元件 | 功能說明 |
|------------|----------|
| `@app.route()` | 定義哪個網址對應哪個函式 |
| `request.method` | 判斷請求方式是 GET 還是 POST |
| `request.form.get(...)` | 取得使用者輸入的表單資料 |
| `app.run(...)` | 啟動網頁伺服器 |

---

這樣你就建立好了一個真正的 Web 登入系統，而且別人可以透過 ngrok 網址直接連進來試用！



# 底下是 n 次的嘗試，是說 LLM 也不是一次成功，我們花了很多時間，vibe_coding_101 ＠deeplearning.ai 中提醒：be patient

In [ ]:
!wget https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz
!tar -xvzf ngrok-v3-stable-linux-amd64.tgz


--2025-03-21 12:51:30--  https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz
Resolving bin.equinox.io (bin.equinox.io)... 75.2.60.68, 13.248.244.96, 35.71.179.82, ...
Connecting to bin.equinox.io (bin.equinox.io)|75.2.60.68|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9372184 (8.9M) [application/octet-stream]
Saving to: ‘ngrok-v3-stable-linux-amd64.tgz’

ngrok-v3-stable-lin 100%[===================>]   8.94M  9.70MB/s    in 0.9s    

2025-03-21 12:51:32 (9.70 MB/s) - ‘ngrok-v3-stable-linux-amd64.tgz’ saved [9372184/9372184]

ngrok


In [ ]:
#!wget -O ngrok.zip https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
#!unzip ngrok.zip


--2025-03-21 12:46:04--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 99.83.220.108, 13.248.244.96, 35.71.179.82, ...
Connecting to bin.equinox.io (bin.equinox.io)|99.83.220.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13921656 (13M) [application/octet-stream]
Saving to: ‘ngrok.zip’

ngrok.zip           100%[===================>]  13.28M  81.2MB/s    in 0.2s    

2025-03-21 12:46:05 (81.2 MB/s) - ‘ngrok.zip’ saved [13921656/13921656]

Archive:  ngrok.zip
  inflating: ngrok                   


In [ ]:
!./ngrok authtoken ....

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
from flask import Flask, request

app = Flask(__name__)

# 假帳號資料
accounts_dict = {
    'user1234': {'password': '5678', 'name': '小明'},
    'user5678': {'password': '1234', 'name': '阿美'}
}

# 簡單登入頁面
login_form = '''
<h2>ATM 登入系統</h2>
<form method="POST">
    帳號：<input type="text" name="username"><br>
    密碼：<input type="password" name="password"><br>
    <input type="submit" value="登入">
</form>
'''

@app.route('/', methods=['GET', 'POST'])
def login():
    if request.method == 'POST':
        username = request.form.get('username')
        password = request.form.get('password')

        if username in accounts_dict and password == accounts_dict[username]['password']:
            return f"<h3>✅ 登入成功！歡迎 {accounts_dict[username]['name']}！</h3>"
        else:
            return "<h3>❌ 帳號或密碼錯誤！</h3>" + login_form
    return login_form

app.run(host='0.0.0.0', port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit


In [ ]:
!./ngrok http 5000


ERROR:  authentication failed: Your account is limited to 1 simultaneous ngrok agent sessions.
ERROR:  You can run multiple simultaneous tunnels from a single agent session by defining the tunnels in your agent configuration file and starting them with the command `ngrok start --all`.
ERROR:  Read more about the agent configuration file: https://ngrok.com/docs/secure-tunnels/ngrok-agent/reference/config
ERROR:  You can view your current agent sessions in the dashboard:
ERROR:  https://dashboard.ngrok.com/agents
ERROR:  
ERROR:  ERR_NGROK_108
ERROR:  https://ngrok.com/docs/errors/err_ngrok_108
ERROR:  


In [ ]:
import threading
import time
from flask import Flask, request
import os

# 安裝 ngrok v3
!wget https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz
!tar -xvzf ngrok-v3-stable-linux-amd64.tgz
!./ngrok config add-authtoken ....

# 啟動 Flask
app = Flask(__name__)

accounts_dict = {
    'user1234': {'password': '5678', 'name': '小明'},
    'user5678': {'password': '1234', 'name': '阿美'}
}

login_form = '''
<h2>ATM 登入系統</h2>
<form method="POST">
    帳號：<input type="text" name="username"><br>
    密碼：<input type="password" name="password"><br>
    <input type="submit" value="登入">
</form>
'''

@app.route('/', methods=['GET', 'POST'])
def login():
    if request.method == 'POST':
        username = request.form.get('username')
        password = request.form.get('password')

        if username in accounts_dict and password == accounts_dict[username]['password']:
            return f"<h3>✅ 登入成功！歡迎 {accounts_dict[username]['name']}！</h3>"
        else:
            return "<h3>❌ 帳號或密碼錯誤！</h3>" + login_form
    return login_form

# 啟動 Flask 的執行緒
def run_flask():
    app.run(host='0.0.0.0', port=5000)

# 啟動 ngrok 的執行緒
def run_ngrok():
    os.system('./ngrok http 5000')

# 開啟兩個 thread
flask_thread = threading.Thread(target=run_flask)
ngrok_thread = threading.Thread(target=run_ngrok)

flask_thread.start()
time.sleep(2)  # 稍微等一下 Flask 啟動好
ngrok_thread.start()


--2025-03-21 12:55:16--  https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz
Resolving bin.equinox.io (bin.equinox.io)... 75.2.60.68, 35.71.179.82, 99.83.220.108, ...
Connecting to bin.equinox.io (bin.equinox.io)|75.2.60.68|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9372184 (8.9M) [application/octet-stream]
Saving to: ‘ngrok-v3-stable-linux-amd64.tgz’

ngrok-v3-stable-lin 100%[===================>]   8.94M  29.3MB/s    in 0.3s    

2025-03-21 12:55:17 (29.3 MB/s) - ‘ngrok-v3-stable-linux-amd64.tgz’ saved [9372184/9372184]

ngrok
Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit


In [ ]:
import threading
import time
from flask import Flask, request
import os
import subprocess

# 安裝 ngrok v3
!wget https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz
!tar -xvzf ngrok-v3-stable-linux-amd64.tgz
!./ngrok config add-authtoken ....

# 啟動 Flask
app = Flask(__name__)

accounts_dict = {
    'user1234': {'password': '5678', 'name': '小明'},
    'user5678': {'password': '1234', 'name': '阿美'}
}

login_form = '''
<h2>ATM 登入系統</h2>
<form method="POST">
    帳號：<input type="text" name="username"><br>
    密碼：<input type="password" name="password"><br>
    <input type="submit" value="登入">
</form>
'''

@app.route('/', methods=['GET', 'POST'])
def login():
    if request.method == 'POST':
        username = request.form.get('username')
        password = request.form.get('password')

        if username in accounts_dict and password == accounts_dict[username]['password']:
            return f"<h3>✅ 登入成功！歡迎 {accounts_dict[username]['name']}！</h3>"
        else:
            return "<h3>❌ 帳號或密碼錯誤！</h3>" + login_form
    return login_form

# 啟動 Flask 的執行緒
def run_flask():
    app.run(host='0.0.0.0', port=5000)

# 改成 subprocess 來跑 ngrok
def run_ngrok():
    ngrok_process = subprocess.Popen(['./ngrok', 'http', '5000'])
    for line in ngrok_process.stdout:
        print(line)

# 開啟兩個 thread
flask_thread = threading.Thread(target=run_flask)
flask_thread.start()

time.sleep(2)  # 等 Flask 啟動
run_ngrok()    # 這裡不需要 threading，直接跑就好


--2025-03-21 13:00:02--  https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz
Resolving bin.equinox.io (bin.equinox.io)... 13.248.244.96, 75.2.60.68, 35.71.179.82, ...
Connecting to bin.equinox.io (bin.equinox.io)|13.248.244.96|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9372184 (8.9M) [application/octet-stream]
Saving to: ‘ngrok-v3-stable-linux-amd64.tgz’

ngrok-v3-stable-lin 100%[===================>]   8.94M  44.9MB/s    in 0.2s    

2025-03-21 13:00:07 (44.9 MB/s) - ‘ngrok-v3-stable-linux-amd64.tgz’ saved [9372184/9372184]

ngrok
Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit


TypeError: 'NoneType' object is not iterable

In [ ]:
import threading
import time
from flask import Flask, request
import os
import subprocess

# 安裝 ngrok v3
!wget https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz
!tar -xvzf ngrok-v3-stable-linux-amd64.tgz
!./ngrok config add-authtoken ....

# 啟動 Flask
app = Flask(__name__)

accounts_dict = {
    'user1234': {'password': '5678', 'name': '小明'},
    'user5678': {'password': '1234', 'name': '阿美'}
}

login_form = '''
<h2>ATM 登入系統</h2>
<form method="POST">
    帳號：<input type="text" name="username"><br>
    密碼：<input type="password" name="password"><br>
    <input type="submit" value="登入">
</form>
'''

@app.route('/', methods=['GET', 'POST'])
def login():
    if request.method == 'POST':
        username = request.form.get('username')
        password = request.form.get('password')

        if username in accounts_dict and password == accounts_dict[username]['password']:
            return f"<h3>✅ 登入成功！歡迎 {accounts_dict[username]['name']}！</h3>"
        else:
            return "<h3>❌ 帳號或密碼錯誤！</h3>" + login_form
    return login_form

# 啟動 Flask 的執行緒
def run_flask():
    app.run(host='0.0.0.0', port=5000)

# 改進 subprocess（開啟 stdout 管道）
def run_ngrok():
    ngrok_process = subprocess.Popen(['./ngrok', 'http', '5000'],
                                     stdout=subprocess.PIPE,
                                     stderr=subprocess.PIPE)

    while True:
        line = ngrok_process.stdout.readline()
        if not line:
            break
        print(line.decode().strip())

# 開啟兩個 thread
flask_thread = threading.Thread(target=run_flask)
flask_thread.start()

time.sleep(2)  # 等 Flask 啟動
run_ngrok()    # 這裡直接跑 ngrok


--2025-03-21 13:44:43--  https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz
Resolving bin.equinox.io (bin.equinox.io)... 99.83.220.108, 13.248.244.96, 75.2.60.68, ...
Connecting to bin.equinox.io (bin.equinox.io)|99.83.220.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9372184 (8.9M) [application/octet-stream]
Saving to: ‘ngrok-v3-stable-linux-amd64.tgz’

ngrok-v3-stable-lin 100%[===================>]   8.94M  16.5MB/s    in 0.5s    

2025-03-21 13:44:44 (16.5 MB/s) - ‘ngrok-v3-stable-linux-amd64.tgz’ saved [9372184/9372184]

ngrok
Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit


KeyboardInterrupt: 

In [ ]:
import threading
import time
from flask import Flask, request
import os

# 安裝 ngrok v3
!wget https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz
!tar -xvzf ngrok-v3-stable-linux-amd64.tgz
!./ngrok config add-authtoken

# 啟動 Flask
app = Flask(__name__)

accounts_dict = {
    'user1234': {'password': '5678', 'name': '小明'},
    'user5678': {'password': '1234', 'name': '阿美'}
}

login_form = '''
<h2>ATM 登入系統</h2>
<form method="POST">
    帳號：<input type="text" name="username"><br>
    密碼：<input type="password" name="password"><br>
    <input type="submit" value="登入">
</form>
'''

@app.route('/', methods=['GET', 'POST'])
def login():
    if request.method == 'POST':
        username = request.form.get('username')
        password = request.form.get('password')

        if username in accounts_dict and password == accounts_dict[username]['password']:
            return f"<h3>✅ 登入成功！歡迎 {accounts_dict[username]['name']}！</h3>"
        else:
            return "<h3>❌ 帳號或密碼錯誤！</h3>" + login_form
    return login_form

# 執行 Flask 的 thread
def run_flask():
    print("🚀 Flask 伺服器啟動中...")
    app.run(host='0.0.0.0', port=5000)

# 執行 ngrok 的 thread
def run_ngrok():
    print("🌐 Ngrok 隧道啟動中...")
    os.system('./ngrok http 5000')

# 啟動兩個執行緒
flask_thread = threading.Thread(target=run_flask)
ngrok_thread = threading.Thread(target=run_ngrok)

flask_thread.start()
time.sleep(3)  # 等待 Flask 啟動穩定
ngrok_thread.start()


--2025-03-21 13:48:11--  https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz
Resolving bin.equinox.io (bin.equinox.io)... 75.2.60.68, 35.71.179.82, 13.248.244.96, ...
Connecting to bin.equinox.io (bin.equinox.io)|75.2.60.68|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9372184 (8.9M) [application/octet-stream]
Saving to: ‘ngrok-v3-stable-linux-amd64.tgz’

ngrok-v3-stable-lin 100%[===================>]   8.94M  12.8MB/s    in 0.7s    

2025-03-21 13:48:12 (12.8 MB/s) - ‘ngrok-v3-stable-linux-amd64.tgz’ saved [9372184/9372184]

ngrok
Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
🚀 Flask 伺服器啟動中...
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit


🌐 Ngrok 隧道啟動中...


In [ ]:
!pip install pyngrok


In [ ]:
from pyngrok import ngrok
public_url = ngrok.connect(5000)
print(public_url)


ERROR:pyngrok.process.ngrok:t=2025-03-21T13:57:01+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2025-03-21T13:57:01+0000 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2025-03-21T13:57:01+0000 lvl=eror msg="terminating with error" obj=app err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your aut

PyngrokNgrokError: The ngrok process errored on start: authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n.

In [ ]:
app.run()


NameError: name 'app' is not defined

In [ ]:
# 安裝 pyngrok
#!pip install pyngrok

# 匯入套件
from flask import Flask, request
from pyngrok import ngrok

# 建立 Flask app
app = Flask(__name__)

# 帳號資料
accounts_dict = {
    'user1234': {'password': '5678', 'name': '小明'},
    'user5678': {'password': '1234', 'name': '阿美'}
}

# HTML 表單
login_form = '''
<h2>ATM 登入系統</h2>
<form method="POST">
    帳號：<input type="text" name="username"><br>
    密碼：<input type="password" name="password"><br>
    <input type="submit" value="登入">
</form>
'''

# 網頁路由
@app.route('/', methods=['GET', 'POST'])
def login():
    if request.method == 'POST':
        username = request.form.get('username')
        password = request.form.get('password')

        if username in accounts_dict and password == accounts_dict[username]['password']:
            return f"<h3>✅ 登入成功！歡迎 {accounts_dict[username]['name']}！</h3>"
        else:
            return "<h3>❌ 帳號或密碼錯誤！</h3>" + login_form
    return login_form

# 啟動 ngrok 隧道，監聽 5000 port
public_url = ngrok.connect(5000)
print(f"🌍 Ngrok 公開網址: {public_url}")

# 啟動 Flask 伺服器
app.run(host='0.0.0.0', port=5000)


ERROR:pyngrok.process.ngrok:t=2025-03-21T13:57:47+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2025-03-21T13:57:47+0000 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2025-03-21T13:57:47+0000 lvl=eror msg="terminating with error" obj=app err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your aut

PyngrokNgrokError: The ngrok process errored on start: authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n.